In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

In [ ]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df = pd.read_csv('Grab Reviews - Phakchum Khumhom.csv')

In [ ]:
df.tail()

,Review_ID,Review
15,16,รอบนี้สั่งยำแบบรสหวาน อร่อยเลยค่ะ แต่เหมือนปริ...
16,17,เหมือนรสชาติไม่ค่อยคงที่มั้ยคะครั้งที่แล้วสั่ง...
17,18,ผัดฟักทองเค็มนำไปหน่อยค่ะ
18,19,สั่งมาครั้งที่ 2 แล้วค่ะ ตั้งแต่ย้ายร้าน เนื่อ...
19,20,ผัดฟักทองรอบนี้เค็มโดดไป ส่วนยำมีไม่ค่อยเข้มข้...


### Tokenize words

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', 'ค่ะ', 'นะคะ', 'รส', 'รสชาติ', 'กิน', 'ทาน','นๆ','น', 'กก', 'ดดด', '(', ')', '/', '+', '.', '1', '2', '3', '4', '5', '6', '7', '8', '9', '55']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ' '
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.tail()

,Review_ID,Review,Review_tokenized
15,16,รอบนี้สั่งยำแบบรสหวาน อร่อยเลยค่ะ แต่เหมือนปริ...,",รอบ,สั่ง,ยำ,หวาน,อร่อย,เหมือน,ปริมาณ,แอบ,ลดลง"
16,17,เหมือนรสชาติไม่ค่อยคงที่มั้ยคะครั้งที่แล้วสั่ง...,",เหมือน,คงที่,สั่ง,เหมือน,ติด,หวาน,รอบ,หวาน,เค็ม"
17,18,ผัดฟักทองเค็มนำไปหน่อยค่ะ,",ผัด,ฟักทอง,เค็ม"
18,19,สั่งมาครั้งที่ 2 แล้วค่ะ ตั้งแต่ย้ายร้าน เนื่อ...,",สั่ง,ย้าย,บ้าน,ตอน,กระแส,สั่ง,ลอง,ตอนนี้,รู้,..."
19,20,ผัดฟักทองรอบนี้เค็มโดดไป ส่วนยำมีไม่ค่อยเข้มข้...,",ผัด,ฟักทอง,รอบ,เค็ม,โดด,ยำ,เข้มข้น,ติด,ขม"


### Create dictionary

In [ ]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['', 'จัดจ้าน', 'จืด', 'ดี', 'ผัด', 'ฟักทอง', 'ยำ', 'หมายถึง', 'อร่อย', 'เผ็ด', 'กลิ่น', 'กะเพรา', 'กับข้าว', 'ขม', 'คน', 'ความคล้ายคลึง', 'คาดหวัง', 'จมูก', 'จับฉ่าย', 'จ้น', 'ชอบ', 'ชิม', 'ซอส', 'ซีอิ้ว', 'ดัน', 'ตำ', 'ตี', 'ทิศทาง', 'นิดนึง', 'นึง', 'น้ำ', 'บวบ', 'ปลายลิ้น', 'ฝา', 'ม', 'มัง', 'รสจัด', 'รอบ', 'รีวิว', 'รู้', 'ลอง', 'ลาบ', 'ล้างน้ำ', 'หน่อไม้', 'อันนี้', 'เขียว', 'เข้มข้น', 'เค็ม', 'เจ้', 'เปรี้ยว', 'เส้น', 'เหมือน', 'เห็ด', 'แกง', 'แกงเลียง', 'แก้ว', 'แม้แต่น้อย', 'โดด', 'โอเค', 'ใส่', 'ไข่เค็ม', 'ข้าว', 'คลี', 'ตุ๋น', 'บุก', 'ส่วนตัว', 'เวิร์ค', 'แปลก', 'เหมือนเดิม', 'ข้าวผัด', 'นุ่ม', 'สั่ง', 'หวาน', 'อาหาร', 'เชื่อม', 'กี้', 'ถั่วเหลือง', 'ถ้วย', 'สูตร', 'เหม็น', 'แน่ใจ', 'แบบนี้', 'แม็ก', 'ไหม', 'ตอน', 'ติด', 'ทำอาหาร', 'พลาสติก', 'รบกวน', 'ระวัง', 'เจอ', 'เศษ', 'ไข่', 'จึ้ง', 'ธรรมดา', 'ปริมาณ', 'ราคา', 'รู้สึก', 'เทียบ', 'แพง', 'คัน', 'ตล๊อดดดดด', 'หลี', 'เด็ด', 'Celery', 'Happy', 'Shot', 'ดีมาก', 'ตก', 'นัว', 'ฝีมือ', 'มีความสุข', 'อารมณ์', 'เพลิน', 

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

### Topic modeling

In [ ]:
num_topics = 20
chunksize = 4000
passes = 20
iterations = 50
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                               alpha='auto', eta='auto', \
                               iterations=iterations, num_topics=num_topics, \
                               passes=passes, eval_every=eval_every)

In [ ]:
vis_data = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds='mmds')

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [ ]:
vis_data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.206563  0.062932       1        1  32.614154
10     0.059483 -0.232346       2        1  13.949390
8     -0.218907 -0.131958       3        1  10.053631
18    -0.151776  0.202215       4        1   7.978151
15     0.173833 -0.165377       5        1   7.963182
17     0.185997 -0.074526       6        1   7.490899
5      0.039848  0.221313       7        1   5.539372
13    -0.090977 -0.152386       8        1   3.805857
0     -0.163161  0.068049       9        1   3.459486
4      0.039207  0.143098      10        1   3.459486
11     0.102137 -0.014169      11        1   2.426465
16    -0.020239  0.008129      12        1   0.142938
1     -0.020220  0.008110      13        1   0.142908
3     -0.020247  0.008130      14        1   0.142825
9     -0.020257  0.008131      15        1   0.138543
12    -0.020257  0.008131      16        1   0.138543
7     -0.020257  0.008131      17        1   0.138543
14    -0.020257  0.008131      18        1   0.138543
6     -0.020257  0.008131      19        1   0.138543
19    -0.020257  0.008131      20        1   0.138543, topic_info=      Term       Freq      Total Category  logprob  loglift
47    เค็ม   6.000000   6.000000  Default  30.0000  30.0000
8    อร่อย  10.000000  10.000000  Default  29.0000  29.0000
51  เหมือน   5.000000   5.000000  Default  28.0000  28.0000
0           15.000000  15.000000  Default  27.0000  27.0000
37     รอบ   7.000000   7.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
25      ตำ   0.002733   1.328024  Topic20  -4.9836   0.3957
26      ตี   0.002733   1.328024  Topic20  -4.9836   0.3957
27  ทิศทาง   0.002733   2.085964  Topic20  -4.9836  -0.0559
28  นิดนึง   0.002733   1.328023  Topic20  -4.9836   0.3957
29     นึง   0.002733   1.328028  Topic20  -4.9836   0.3957

[1038 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.258691         
0         2  0.194018         
0         3  0.064673         
0         4  0.064673         
0         5  0.194018         
...     ...       ...      ...
92        6  0.872584      ไข่
60        1  0.753007  ไข่เค็ม
128       7  0.913662     ไลน์
129       7  0.913662       ไว
83        2  0.805436      ไหม

[226 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 11, 9, 19, 16, 18, 6, 14, 1, 5, 12, 17, 2, 4, 10, 13, 8, 15, 7, 20])